In [ ]:
%pip install crewai_tools crewai -U

In [ ]:
%pip install --upgrade crewai crewai_tools

In [ ]:
pip show crewai

In [ ]:
%pip install -U langchain-google-genai


In [ ]:
# Initialize all your components
from crewai import Crew, Process, LLM
#import agentops
from dotenv import load_dotenv
import os
import json
#from datetime import datetime
# Import your other agents from their respective files
from Requirement_Scout_Agent1 import Extractor_agent

#Key Needed
load_dotenv()
api_key = os.getenv("GEMINI_API_KEY")

if not api_key:
    raise ValueError("لم يتم العثور على GOOGLE_API_KEY. تأكد من إضافته لملف .env")

#LLM

my_llm = LLM(
    model="gemini/gemini-1.0-flash", 
    api_key=os.environ.get("GOOGLE_API_KEY")
)

#main.py
def main():
    #from crewai.knowledge.source.string_knowledge_source import StringKnowledgeSource

    # Initialize the search recommender
    output_dir=r"./Capstone_Project_SDAIA/src/outputs"
    Extractor= Extractor_agent(llm=my_llm, output_dir=output_dir)
    

    #context_dependency
   
    #knowledge_sources
    """about_file = "ohay is a company that provides AI solutions to help websites refine their search and recommendation systems."
    file_context = StringKnowledgeSource(
        content=about_file
    )"""

    # Create the crew
    crew = Crew(
        agents=[
            Extractor.get_agent,
    
        ],
        tasks=[
            Extractor.get_task,
            
        ],
        process=Process.sequential,
        max_rpm=1,
        
        #knowledge_sources=[file_context],  # Make sure company_context is defined
        #memory=True
    )
    
   
    result = crew.kickoff()

    print("=== RESULTS ===")
    print(result)
    return result

if __name__ == "__main__":
    main()

In [ ]:
load_dotenv()
api_key = os.getenv("GOOGLE_API_KEY")

if not api_key:
    raise ValueError("لم يتم العثور على GOOGLE_API_KEY. تأكد من إضافته لملف .env")


In [ ]:
%pip install "unstructured[md]"

In [ ]:
import os
from agentic_doc.parse import parse

# 1. تحديد المسارات
pdf_path = r"C:\Users\user\OneDrive - University of Prince Mugrin\سطح المكتب\Capstone_Project_SDAIA\src\data\request1.pdf"
output_folder = r"C:\Users\user\OneDrive - University of Prince Mugrin\سطح المكتب\Capstone_Project_SDAIA\src\data\processed"

# إنشاء المجلد إذا لم يكن موجوداً
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# 2. عملية التحويل
print("جاري تحويل الملف من PDF إلى MD...")
results = parse([pdf_path])

# 3. حفظ النتيجة كملف .md
# نفترض أن results عبارة عن قائمة كائنات، نأخذ النص من العنصر الأول
md_filename = "request1_raw.md"
md_output_path = os.path.join(output_folder, md_filename)

with open(md_output_path, "w", encoding="utf-8") as f:
    # ملاحظة: تأكد من مخرجات مكتبة agentic_doc، عادة النص يكون في .text أو .markdown
    f.write(results[0].markdown) 

print(f"✅ تم حفظ الملف الخام في: {md_output_path}")

clean 

In [ ]:
import re
from unstructured.partition.md import partition_md
from unstructured.cleaners.core import clean_extra_whitespace

def auto_clean_with_unstructured(file_path):
    # 1. تقسيم الملف إلى عناصر
    elements = partition_md(filename=file_path)

    cleaned_text = []

    for element in elements:
        text = element.text.strip()
        
        # --- الفلترة بناءً على بداية النص (أوصاف الصور واللوقو) ---
        image_keywords = [
            "Summary :", "logo:", "Visible Elements :", "Placement & Dimensions :", 
            "Analysis :", "Graphic Elements :", "Text Elements :", "Design & Placement :", 
            "Design & Layout :", "Layout :", "Design Elements :", "Design Details :", 
            "Text Fields :", "Colour Palette :", "Spatial Relationships :","Dimensions & Placement :","/25, 4:29 PM"
        ]
        
        if any(text.startswith(key) for key in image_keywords):
            continue 

        # --- الفلترة بناءً على وجود كلمات معينة (الترويسات والتذييلات) ---
        excluded_headers = [
            "المملكة العربية السعودية", "كراسة الشروط", "تاريخ الإصدار", 
            "tenders.etimad.sa", "المعتمد بموجب قرار وزير المالية","اسم المنافسة: ","رقم الكراسة: ","تاريخ طرح الكراسة :"
        ]
        if any(key in text for key in excluded_headers):
            continue

        # --- تصحيح حذف أرقام الصفحات والروابط باستخدام Regex ---
        # حذف أرقام الصفحات مثل 31/37
        text = re.sub(r'\d+/\d+', '', text)
        
        # حذف أي رابط يبدأ بـ http (روابط منصة اعتماد)
        text = re.sub(r'https?://\S+', '', text)

        # 4. تنظيف الفراغات الناتجة عن الحذف
        text = clean_extra_whitespace(text)

        # إضافة النص النظيف فقط إذا لم يصبح فارغاً بعد الحذف
        if text.strip():
            cleaned_text.append(text)

    return "\n\n".join(cleaned_text)

# --- التشغيل ---
input_file = r"C:\Users\user\OneDrive - University of Prince Mugrin\سطح المكتب\Capstone_Project_SDAIA\src\data\processed\request1_raw.md"
final_content = auto_clean_with_unstructured(input_file)

with open("request1_final_clean3.md", "w", encoding="utf-8") as f:
    f.write(final_content)

print("✅ تم تنظيف الملف بنجاح وحذف أوصاف الصور والروابط وأرقام الصفحات!")

✅ تم تنظيف الملف بنجاح وحذف أوصاف الصور والروابط وأرقام الصفحات!


In [ ]:
import re
import os

def clean_rfp_markdown(input_file_path, output_file_path):
    with open(input_file_path, 'r', encoding='utf-8') as file:
        content = file.read()

    # 1. حذف الروابط المتكررة الخاصة بمنصة اعتماد (التذييل)
    # يبحث عن أي سطر يبدأ بـ https://tenders.etimad.sa
    content = re.sub(r'https://tenders\.etimad\.sa\S+', '', content)

    # 2. حذف أرقام الصفحات المتكررة (مثل 2/37, 3/37...)
    content = re.sub(r'\d+/\d+', '', content)

    # 3. حذف التاريخ والوقت الذي يظهر في أعلى الصفحة (مثلاً: 5/1/25, 4:29 PM)
    content = re.sub(r'\d+/\d+/\d+,\s+\d+:\d+\s+[AP]M', '', content)

    # 4. حذف النصوص الثابتة المتكررة (كراسة الشروط والمواصفات، المملكة العربية السعودية، إلخ)
    static_texts = [
        "كراسة الشروط والمواصفات",
        "المملكة العربية السعودية",
        "إسم الإدارة",
        "إسم النموذج",
        "رقم النسخة",
        "تاريخ الإصدار",
        "رقم الكراسة",
        "بيانات مكتب تحقيق الرؤية"
    ]
    for text in static_texts:
        content = content.replace(text, '')

    # 5. حذف الصور/اللوقو (في المارك داون تكون عادة على شكل ![image](...))
    content = re.sub(r'!\[.*?\]\(.*?\)', '', content)

    # 6. تنظيف الفراغات الزائدة والأسطر الفارغة الكثيرة التي نتجت عن الحذف
    content = re.sub(r'\n\s*\n', '\n\n', content)
    
    # حفظ الملف النظيف
    with open(output_file_path, 'w', encoding='utf-8') as file:
        file.write(content.strip())

    print(f"✅ تم تنظيف الملف وحفظه في: {output_file_path}")

# --- طريقة الاستخدام ---
input_path = r"C:\path\to\your\original_file.md"
output_path = r"C:\path\to\your\cleaned_file.md"

clean_rfp_markdown(input_path, output_path)

In [ ]:

def read_toc_only(file_path, limit_lines=150):
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = [next(f) for _ in range(limit_lines)]
    return "".join(lines)

# ثم نمرر هذا النص للـ Task مباشرة
toc_content = read_toc_only(r"C:\Users\user\OneDrive - University of Prince Mugrin\سطح المكتب\Capstone_Project_SDAIA\src\Agents\src\data.md